In [1]:
import pandas as pd
import os
import email
import time
import csv
import sys
from sklearn.model_selection import train_test_split
import math
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
from joblib import Parallel, delayed
import multiprocessing

#unique delim to separate columns without messing up content
DELIMETER = chr(255)

#needed because a few of the emails were too large for the default csv cell size
csv.field_size_limit(sys.maxsize)
#set paths (currently pathing for remote AWS EC2 Instance)
root_path = '/home/ubuntu/ssl/notebooks/data/maildir'
saved_file = '/home/ubuntu/ssl/notebooks/processed enron emails.csv'
testing_batch = False

In [2]:
#steps through directory and loads every email in every folder into returned df
def process_emails_into_df(origin, destination, testbool):
    #safety check to see if the file has been created so I don't overwrite a finished file
    already_processed = os.path.isfile(destination) 

    #if no file already exists start the writing process
    if not already_processed:

        #list for holding file paths
        list_of_files = []    

        #get the path for all the email files
        for path, subdirs, files in os.walk(origin):
            for name in files:
                list_of_files.append(os.path.join(path, name))

        email_count = len(list_of_files)
        #caluculate 1% of the number of files, used for progress report printing.
        onepcofemail = (email_count//100)



        #create dataframe to store data to be written
        emails = pd.DataFrame()
        print('Starting. {}'.format(time.strftime("%I:%M:%S")))

        #counter for testing batches
        x = 0

        #iterrate through all the files found
        for i, myfile in enumerate(list_of_files):

            #if running a testing session to break before reading all file to inspect the saved csv
            if x > 6000 and testbool: break

            #read individual email file
            with open(myfile, 'r', encoding='utf-8', errors='replace') as filepath:
                #read in the email
                message=email.message_from_string(filepath.read())

                #save the important components
                emails.loc[i,'Message-ID']=message['Message-ID']
                emails.loc[i,'from']=message['from']
                emails.loc[i,'subject']=message['subject']
                emails.loc[i,'to']=message['to']
                emails.loc[i,'cc']=message['cc']
                emails.loc[i,'bcc']=message['Bcc']
                emails.loc[i,'date']=message['date']
                emails.loc[i,'file']=message['X-FileName']

                #check that the body is one or multiple sections and save it
                if message.is_multipart():
                    string = ''
                    for payload in message.get_payload():
                        print(payload.get_payload())
                        string = string + payload.get_payload()
                        emails.loc[i,'body'] = string
                else:
                    emails.loc[i,'body'] =  message.get_payload()

            #append entry to the csv file
            with open(destination, 'a') as f:
                #if this is the first entry use the headers, if not, dont.True
                if i == 0: emails.to_csv(f, header=emails.columns, index = False, sep=DELIMETER)
                else: emails.to_csv(f, header=None, index = False, sep=DELIMETER)

            #clear the dataframe to save memory
            emails = pd.DataFrame()        

            #print statement updating progress report
            if i % onepcofemail == 0: 
                print('{}% finished. {}'.format(i//onepcofemail, time.strftime("%I:%M:%S")), end="\r")

            #itterate counter for testing batches
            x = x+1


        #process complete
        print('Done! Exported {} lines to CSV {}'.format(x, time.strftime("%I:%M:%S")))

    #prints if the file already exists and there is no need to process the individual emails into a csv
    else: 
        #open csvs of all emails
        print('**********There is already a Process CSV of All Emails, loading Now************', end="\r")
        all_emails = pd.read_csv(saved_file, sep=DELIMETER, engine='python')
        print('***Processed Emails into DF Loaded From CSV, Delete File to Process Again******')
        return all_emails        

#function that filters out non enron originating email addresses       
def filter_unwanted_senders(dataframe, new_file_dest):
    path='/home/ubuntu/ssl/notebooks/' + new_file_dest
    
    #check to see if file has already been processed and saved
    if not os.path.isfile(path): 
        
        #get a list of unique email senders
        unique_senders_inc_small = dataframe['from'].unique()
        
        #create an empty df to holde data
        filtered_df = pd.DataFrame(columns = dataframe.columns)
        
        #iterate through all the unique senders
        for i, sender in enumerate(unique_senders_inc_small):
            temp = dataframe[dataframe['from'] == sender]
            
            #prints progress report
            print('{}% finished filtering unwanted senders            '.format(round(i/len(
                                                                            unique_senders_inc_small)
                                                                                     *100,2)) , end="\r")

            #tests and saves senders who have enron emails
            if '@enron.com' in sender:
                filtered_df = filtered_df.append(temp, ignore_index = True)
        
        #save new list of senders
        filtered_df.to_csv(path, sep=DELIMETER, index = False)
    
    #if it has already been done don't do it again
    else: 
        print('*********Previously Filtered Senders List. Loading Old File!*********'.format(new_file_dest),
                                                                                              end="\r")
        filtered_df = pd.read_csv(path, sep=DELIMETER, engine='python')
        print('****To Reprocess Filtered Senders Delete {}.  Loaded Old File!*******'.format(new_file_dest))
    filtered_df = filtered_df.dropna(subset = ['to', 'cc', 'bcc'], how = 'all')
    filtered_df = filtered_df.dropna(subset = ['body'])    
    return filtered_df

#makes a list of all senders and a lists all the recipients they send to in next column
def get_sender_recip_list(dataframe):
    new_file = '/home/ubuntu/ssl/notebooks/sender_recip_df_list'
    
    if not os.path.isfile(new_file):
    
        #empty df for leading up addresses
        sender_by_recip = pd.DataFrame(columns = ['sender','recipient', 'booksize'])

        #list of unique senders
        unique_senders = dataframe['from'].unique()

        #arrays for holding values to be stored
        senders = []
        recipients = []
        addressbooksize = []

        #itterate through each unique email writer
        for i, sender in enumerate(unique_senders):

            #make a list of the samples from this itterations sender
            users_emails = dataframe[dataframe['from'] == sender]

            all_recips = users_emails['to'].tolist() + users_emails['cc'].tolist() + users_emails['bcc'].tolist() 

            if '<' in sender:
                sender = re.search('<.(.*)>', sender)
                sender = sender.group(1)
            

            namelist = []
            for l in all_recips:
                if not isinstance(l, str): pass
                else: 
                    l = l.replace('\t','').replace('\n','')
                    list_of_names = l.split(',')
                    for name in list_of_names:
                        
                        if '<' in name:
                            result = re.search('<.(.*)>', name)
                            name = result.group(1)
                            name = name.replace('"', '')
                            name = name.replace("'", '')
                            name = name.replace("..", '.')
                            name = name.strip(' ')
                            namelist.append(name)
                        
                        else:
                            name = name.replace('"', '')
                            name = name.replace("'", '')
                            name = name.strip(' ')
                            name = name.replace("..", '.')
                            namelist.append(name)
                        


            #name = [item for sublist in name for item in sublist]
            unique_recips = list(set(namelist))
            for name in unique_recips:
                recipients.append(name)
                senders.append(sender)
                addressbooksize.append(len(unique_recips))
            

            print('{}% of sender + recip dataframe completed        '.format(
                                                                        round(i/len(unique_senders)*100,2))
                                                                        , end="\r")

        sender_by_recip['sender'] = senders
        sender_by_recip['recipient'] = recipients
        sender_by_recip['booksize'] = addressbooksize
        print('********************* Sender Recip Dataframe Completed ******************')
        sender_by_recip.to_csv(new_file, sep=DELIMETER, index = False)
        return sender_by_recip
    else:
        print('********************* Sender Recip Already Processed ********************', end="\r")
        sender_by_recip = pd.read_csv(new_file, sep=DELIMETER, engine = 'python')
        print('************ Sender Recip Dataframe with Lists CSV Loaded ***********')
        return sender_by_recip


def remove_strang_problematic_addesses(s_r_pair_list):
    
    #this one was VERY problematic to parse, maybe best to skip this sender
    s_r_pair_list = s_r_pair_list[s_r_pair_list['sender'] != 'enron.announcements@enron.com']
    
    s_r_pair_list = s_r_pair_list[s_r_pair_list['sender'] != '40enron@enron.com']

    
    #there were some junk addresses that were clearly just in error, lets drop those
    bad_index = s_r_pair_list[s_r_pair_list['recipient'].str.contains('@enron@enron.com')].index

    #do the actual dropping
    s_r_pair_list.drop(bad_index, inplace = True)
    del bad_index
    
    #there were some junk addresses that were clearly just in error, lets drop those
    bad_index = s_r_pair_list[s_r_pair_list['recipient'].str.contains('/')].index

    #do the actual dropping
    s_r_pair_list.drop(bad_index, inplace = True)
    del bad_index
    
    #there were some junk addresses that were clearly just in error, lets drop those
    bad_index = s_r_pair_list[s_r_pair_list['recipient'].str.contains('&')].index

    #do the actual dropping
    s_r_pair_list.drop(bad_index, inplace = True)
    del bad_index
    
    #there were some junk addresses that were clearly just in error, lets drop those
    bad_index = s_r_pair_list[s_r_pair_list['recipient'] == '<'].index

    #do the actual dropping
    s_r_pair_list.drop(bad_index, inplace = True)
    del bad_index
    
    #get rid of peoples typos sending to @enron.com
    s_r_pair_list = s_r_pair_list[s_r_pair_list['recipient'] != '@enron.com']
    
    #reset index values after removing elements from the process
    s_r_pair_list.index = range(len(s_r_pair_list.index))
    
    return s_r_pair_list

def email_exchange_count(pair_list, emails):
    new_list = pd.DataFrame()
    new_file = '/home/ubuntu/ssl/notebooks/emailcountsadded'
    
    senders = []
    recipient = []
    addressbook = []
    numemails = []
    
    if not os.path.isfile(new_file):
        for index, row in pair_list.iterrows():
                #current sender
                from_ = row['sender']
                #current recipient
                to_ = row['recipient']

                #select all eamil bodies from and to current selection
                combo_s_r = emails[(emails['from']== from_) & ((emails['to'].isin(to_)) | (emails['cc'].isin(to_)) | (emails['bcc'].isin(to_)))]['body']

                
                senders.append(from_)
                recipient.append(to_)
                addressbook.append(row['booksize'])
                numemails.append(len(combo_s_r))
                
                if(index%(round(len(pair_list)/10000,0)) == 0):
                    print('{}% done Time: {}                            '.format((round(index/
                                                                    len(pair_list),4)*100),
                                                                    time.strftime("%I:%M:%S")), end="\r")
        
        new_list['sender'] = senders
        new_list['recipient'] = recipient
        new_list['addressbooksize'] = addressbook
        new_list['num_emails'] = numemails

        
        new_list.to_csv(new_file, sep=DELIMETER, index = False)
        return new_list
    
    else:
        
        #let user know process was previously completed and open previously processed file
        print('Counts were previously calculated.  Opening saved File', end = '\r')
        new_list = pd.read_csv(new_file, sep=DELIMETER, engine = 'python')
        print('*********EmailCounts  file Opened *******************')
        return new_list    
    
#calculate the mean tf_idf score for a series of texts
def tfidf(txt, vectorizer):
    try: 
        X = vectorizer.fit_transform(txt)
        Y = list(X.tocoo().data)
        Y = [i/sum(Y) for i in Y]
    except ValueError:
        Y = [0]
    return Y


def similarity(l1, l2): # padding the uneven vectors by zero and compute cosine similarity 
    diff = len(l1) - len(l2)
    if (diff < 0):
        l1 = l1 + np.abs(diff)*[0]
    else:
        l2 = l2 + np.abs(diff)*[0]
    similarityscore = cosine_similarity(l1, l2)
    return similarityscore[0][0]



#add the tf_idf score to a dataframe listing senders paired with recipients from a dataframe of emails
def add_tf_score(combo_list, emails, tfidfv, test_set):
    
    #saved file location
    new_file = '/home/ubuntu/ssl/notebooks/scored_w_mean_idf'
    
    #if already completed calculations before do not recalculate or overwrite file
    if not os.path.isfile(new_file):
        
        #list for holding mean tf_idf scores
        senders = []
        recievers = []
        tf_idf_body =[]
        
        #iterate through all sender/recipient combos in the dataframe
        for index, row in combo_list.iterrows():
            #current sender
            from_ = row['sender']
            #current recipient
            to_ = row['recipient']
            
#             #select all eamil bodies from and to current selection
            combo_s_r = emails[(emails['from']== from_) & ((emails['to'].str.contains(to_)) | (emails['cc'].str.contains(to_)) | (emails['bcc'].str.contains(to_)))]['body']

        
            print('{}% done Time: {}, From: {}, To: {}. Entries: {}                            '.format((round(index/
                                                            len(combo_list),4)*100),
                                                            time.strftime("%I:%M:%S"), from_, to_,len(combo_s_r)), end="\r")

            #calculate mean tf_idf score
            tf_idf_mean = tfidf(combo_s_r, tfidfv)
            

            #append score to list
            senders.append(from_)
            recievers.append(to_)
            tf_idf_body.append(tf_idf_mean)
            
            
            ########################################################################
            
            '''test_set['email_idf'] = tfidfv.transform(test_set['body'])
            cosine_scores = pd.DataFrame()
            
            from sklearn.metrics.pairwise import cosine_similarity
            for email in test_set.iterrows():
                emailidf = email['body']
                print(emailidf)
                '''
            ########################################################################

        #append tf_idf scores to dataframe in new column
        calced_tfidf_scores = pd.DataFrame()
        calced_tfidf_scores['from'] = senders
        calced_tfidf_scores['to'] = recievers
        calced_tfidf_scores['tf_idf_mean'] = tf_idf_body
        
        #save file with unique delimeter
        calced_tfidf_scores.to_csv(new_file, sep=DELIMETER, index = False)
        return calced_tfidf_scores   
    else:
        
        #let user know process was previously completed and open previously processed file
        print('Scores were previously calculated.  Opening saved File')
        calced_tfidf_scores = pd.read_csv(new_file, sep=DELIMETER, engine = 'python')
        print('*********TF_IDF scores file Opened *******************')
        return calced_tfidf_scores  

In [3]:
#read in all files to process emails into dataframe
all_emails = process_emails_into_df(root_path, saved_file, testing_batch)

#filter out the non enron email senders
wanted_emails = filter_unwanted_senders(all_emails, 'small_senders_filtered')

#compile a dataframe of all senders and all unique recipients to that sender
senders_to = get_sender_recip_list(wanted_emails)

#some final cleaning of addresses that cause problems or that are clearly just junk
clean_pair_list = remove_strang_problematic_addesses(senders_to)

#count the number of emails sent from each sender to each recipient
email_counts_added = email_exchange_count(clean_pair_list, wanted_emails)

smaller_sr_list = email_counts_added[email_counts_added['num_emails'] > 50]

smaller_sr_list = smaller_sr_list[smaller_sr_list['sender']!='outlook.team@enron.com']

***Processed Emails into DF Loaded From CSV, Delete File to Process Again******
****To Reprocess Filtered Senders Delete small_senders_filtered.  Loaded Old File!*******
************ Sender Recip Dataframe with Lists CSV Loaded ***************
*********EmailCounts  file Opened *******************e


In [4]:
tfidfv = TfidfVectorizer(min_df=1)

In [50]:
corpus = smaller_sr_list[smaller_sr_list['sender']=='david.forster@enron.com']
corpus.index = range(len(corpus.index))

In [16]:
X_train, X_test = train_test_split(wanted_emails, test_size = 0.25)

In [17]:
davids_email_cosine = pd.DataFrame

In [25]:
X_train.shape

(272685, 10)

In [ ]:
print('Starting {}'.format(time.strftime("%I:%M:%S")))
tfidfv = TfidfVectorizer(min_df=1)
centroid_scores = add_tf_score(corpus, X_train, tfidfv, X_test)
print('Finished {}'.format(time.strftime("%I:%M:%S")))

Starting 09:25:07


In [ ]:
for row in X_train_david.iterrows():

In [134]:
print('Starting {}'.format(time.strftime("%I:%M:%S")))
scores = add_tf_score(smaller_sr_list, wanted_emails)
print('Finished {}'.format(time.strftime("%I:%M:%S")))

In [ ]:
print('Starting {}'.format(time.strftime("%I:%M:%S")))
scores = add_tf_score(clean_pair_list, X_train)
print('Finished {}'.format(time.strftime("%I:%M:%S")))

In [ ]:
def add_tf_score(combo_list, emails, tfidfv):
    
    #saved file location
    new_file = '/home/ubuntu/ssl/notebooks/scored_w_mean_idf'
    
    #if already completed calculations before do not recalculate or overwrite file
    if not os.path.isfile(new_file):
        
        #list for holding mean tf_idf scores
        senders = []
        recievers = []
        tf_idf_body =[]
        
        #iterate through all sender/recipient combos in the dataframe
        for index, row in combo_list.iterrows():
            #current sender
            from_ = row['sender']
            #current recipient
            to_ = row['recipient']
            
            #select all eamil bodies from and to current selection
            combo_s_r = emails[(emails['from']== from_) & ((emails['to'].str.contains(to_)) | (emails['cc'].str.contains(to_)) | (emails['bcc'].str.contains(to_)))]['body']
            

            print('{}% done Time: {}, From: {}, To: {}. Entries: {}                            '.format((round(index/
                                                            len(combo_list),4)*100),
                                                            time.strftime("%I:%M:%S"), from_, to_,len(combo_s_r)), end="\r")

            #calculate mean tf_idf score
            tf_idf_mean = tfidf(combo_s_r, tfidfv)

            #append score to list
            senders.append(from_)
            recievers.append(to_)
            tf_idf_body.append(tf_idf_mean)



        #append tf_idf scores to dataframe in new column
        calced_tfidf_scores = pd.DataFrame()
        calced_tfidf_scores['from'] = senders
        calced_tfidf_scores['to'] = recievers
        calced_tfidf_scores['tf_idf_mean'] = tf_idf_body
        
        #save file with unique delimeter
        calced_tfidf_scores.to_csv(new_file, sep=DELIMETER, index = False)
        return calced_tfidf_scores   
    else:
        
        #let user know process was previously completed and open previously processed file
        print('Scores were previously calculated.  Opening saved File')
        calced_tfidf_scores = pd.read_csv(new_file, sep=DELIMETER, engine = 'python')
        print('*********TF_IDF scores file Opened *******************')
        return calced_tfidf_scores


In [ ]:
#Testing output of code with parallel processing
testsample = list_of_s_r[list_of_s_r['sender'] == 'leslie.hansen@enron.com']
print('Starting {}'.format(time.strftime("%I:%M:%S")))
scores = add_tf_score_v2(testsample, wanted_emails)
print('Finished {}'.format(time.strftime("%I:%M:%S")))